In [21]:
import numpy as np
import pandas as pd
import scripts as sc
import seaborn as sns
import matplotlib.pyplot as plt

prefix = 'r'

pd.options.display.max_rows

Matplotlib is building the font cache; this may take a moment.


500

## Abertura dos arquivos

In [3]:
dataAnswers = pd.read_csv('data/{}/Answers.csv'.format(prefix), encoding='latin-1')
dataQuestions = pd.read_csv('data/{}/Questions.csv'.format(prefix), encoding='latin-1')
dataTags = pd.read_csv('data/{}/Tags.csv'.format(prefix))

len(dataAnswers), len(dataQuestions), len(dataTags)

(250788, 189930, 319375)

## Remover todos os valores nulos do conjunto de dados

In [4]:
dataAnswers = dataAnswers.dropna()
dataQuestions = dataQuestions.dropna()
dataTags = dataTags.dropna()

len(dataAnswers), len(dataQuestions), len(dataTags)

(250000, 188729, 319212)

## Informações sobre os conjuntos de dados

In [5]:
dataAnswers.info(), dataQuestions.info(), dataTags.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 250000 entries, 0 to 250787
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Id                250000 non-null  int64  
 1   OwnerUserId       250000 non-null  float64
 2   CreationDate      250000 non-null  object 
 3   ParentId          250000 non-null  int64  
 4   Score             250000 non-null  int64  
 5   IsAcceptedAnswer  250000 non-null  bool   
 6   Body              250000 non-null  object 
dtypes: bool(1), float64(1), int64(3), object(2)
memory usage: 13.6+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 188729 entries, 0 to 189929
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Id            188729 non-null  int64  
 1   OwnerUserId   188729 non-null  float64
 2   CreationDate  188729 non-null  object 
 3   Score         188729 non-null  int64  
 4   Title  

(None, None, None)

## Adição das colunas: ano, mes e dia

In [6]:
dateQuestion = sc.getDateFormat(dataQuestions['CreationDate'].values)
dateAnswer = sc.getDateFormat(dataAnswers['CreationDate'].values)

dataQuestions['ano'] = dateQuestion[0][0][0]
dataQuestions['mes'] = dateQuestion[0][0][1]
dataQuestions['dia'] = dateQuestion[0][0][2]

dataAnswers['ano'] = dateAnswer[0][0][0]
dataAnswers['mes'] = dateAnswer[0][0][1]
dataAnswers['dia'] = dateAnswer[0][0][2]

## Tamanho dos conjuntos por ano

In [7]:
print('\t\tQuestões:\t\tPerguntas:')
for ano in range(2008, 2016+1):
    print('ano: {}\ttamanho: {}\t\ttamanho: {}'.format(ano, len(dataQuestions.loc[(dataQuestions['ano'] == ano)]), len(dataAnswers.loc[(dataAnswers['ano'] == ano)])))

		Questões:		Perguntas:
ano: 2008	tamanho: 4		tamanho: 12
ano: 2009	tamanho: 455		tamanho: 1145
ano: 2010	tamanho: 2178		tamanho: 4541
ano: 2011	tamanho: 5604		tamanho: 10665
ano: 2012	tamanho: 11541		tamanho: 18173
ano: 2013	tamanho: 20651		tamanho: 30652
ano: 2014	tamanho: 29218		tamanho: 40050
ano: 2015	tamanho: 38388		tamanho: 49155
ano: 2016	tamanho: 42019		tamanho: 51928


## Total de questões para cada ano

In [8]:
dataQuestions['ano'].value_counts()

2016    42019
2017    38671
2015    38388
2014    29218
2013    20651
2012    11541
2011     5604
2010     2178
2009      455
2008        4
Name: ano, dtype: int64

## Estamos considerando que um usuário não novato (usuario veterano), os usuários do ano de 2008 e 2009, já que no ano de 2008 há apenas 4 questões.

In [9]:
veterans = dataQuestions.loc[(dataQuestions['ano'] == 2008) | (dataQuestions['ano'] == 2009), 'OwnerUserId'].unique().tolist()

len(veterans)

197

In [10]:
dropVeteran = False

if dropVeteran:
    for idVeteran in dataQuestions.loc[(dataQuestions['ano'] == 2008) | (dataQuestions['ano'] == 2009), 'OwnerUserId'].unique().tolist():
        
        for idx in dataQuestions.loc[dataQuestions['OwnerUserId'] == idVeteran].index:
            dataQuestions = dataQuestions.drop(index=idx)

len(dataQuestions)

188729

## Total de novatos nos anos

In [11]:
dataQuestions['Body'] = dataQuestions['Body'].str.lower()

In [12]:
data = []
libTermsR = [
    ['dplyr'], 
    ['ggplot2'], 
    ['data.table'],
    ['tidyr'],
    ['Shiny'],
    ['knitr'],
    ['mlr3'],
    ['XGBoost'],
    ['Caret'],
    ['Esquisse'],
    ['Brilhante'],
    ['mlr3'],
    ['lubridate'],
    ['H20'],
    ['mlr'],
    ['Prophet'],
    ['Plotly'],
    ['lattice'],
    ['jsonlite'],
    ['slidify'],
]

for libs in libTermsR:
    dataTopic = []
    qtdNewComers = 0
    qtdTotal = 0
    
    for lib in libs:
        dataQuestions['lib'] = dataQuestions['Body'].map(lambda val: val.count(lib)>0)
        
        if len(dataTopic) > 1:
            dataTopic.loc[dataTopic['lib'] == False, 'lib'] = dataQuestions.loc[dataQuestions['lib'] == True]
        else:
            dataTopic = dataQuestions.loc[dataQuestions['lib'] == True]

    for ano in range(min(dataQuestions['ano'])+1, max(dataQuestions['ano'])+1):
        dataAno = dataTopic.loc[dataTopic['ano'] == ano]

        if len(dataAno) < 1:

            for mes in range(1, 13):
                data.append([prefix, libs[0], ano, mes, qtdNewComers, qtdTotal])
        
        else:

            for mes in range(min(dataQuestions['mes']), max(dataQuestions['mes'])+1):
                dataMes = dataAno.loc[dataAno['mes'] == mes]
                
                if len(dataMes) < 1:
                    data.append([prefix, libs[0], ano, mes, qtdNewComers, qtdTotal])
                
                else:
                    for question in dataMes.values:
                        
                        if question[1] not in veterans:
                            qtdNewComers += 1
                        
                        qtdTotal += 1
                    
                    data.append([prefix, libs[0], ano, mes, qtdNewComers, qtdTotal])

dataLanguage = pd.DataFrame(data, columns=['Language', 'Lib', 'Year', 'Month', 'NewCommers', 'Total'])